# Latent Dirichlet Allocation & Topic Models

In this notebook, we describe the basics of LDA for learning topics of text corpora.

In [1]:
import numpy as np
import scipy.sparse

### "Trivial" example

Let's build a trivial data set consisting of just a few very short "documents":

In [5]:
# The documents themselves:
docs    = ['Human machine interface for ABC computer applications',
           'A survey of user opinion of computer system response time',
           'The EPS user interface management system',
           'System and human system engineering testing of EPS',
           'Relation of user perceived response time to error measurement',
           'The generation of random, binary, ordered trees',
           'The intersection graph of paths in trees',
           'Graph minors IV: Widths of trees and well-quasi-ordering',
           'Graph minors: A survey']

# Normally, we would remove stop words, etc; 
# here we'll just use a pre-constructed "interesting word" list
vocab_list = ['human','interface','computer','user','system','response','time', 
         'EPS','survey','trees','graph','minors']

vocab = {w.lower():i for i,w in enumerate(vocab_list)}
words = {i:w.lower() for i,w in enumerate(vocab_list)}
print(vocab)

{'human': 0, 'interface': 1, 'computer': 2, 'user': 3, 'system': 4, 'response': 5, 'time': 6, 'eps': 7, 'survey': 8, 'trees': 9, 'graph': 10, 'minors': 11}


It's useful to define some text helper functions to filter the documents:

In [8]:
stopwords = 'the and was for that you with have are this from can which has were don'.split(' ')
# if exist stopwords.txt: load & append to stopwords
def filter_text(text):
    import re
    words = re.sub(r'[^\w ]',' ',text)
    words = map(lambda w: w.lower(), words.split(' '))
    words = filter(lambda w: w not in stopwords and len(w)>2, words)
    return words
    
print(list(filter_text('This will filter out all the unwanted words, eh?')))

['will', 'filter', 'out', 'all', 'unwanted', 'words']


Now, we translate the data into index form.  Each word in the corpus is one data point, where data point i is represented by a document ID "d[i]" indicating which document it came from, and a word ID "w[i]" indicating which word in the vocabulary it was.

In [9]:
w = np.array([vocab[j] for i,txt in enumerate(docs) for j in filter_text(txt) if j in vocab ],dtype=int)
d = np.array([i for i,txt in enumerate(docs) for j in filter_text(txt) if j in vocab ],dtype=int)
D = max(d)+1
W = max(w)+1
N = len(d)

In [10]:
w

array([ 0,  1,  2,  8,  3,  2,  4,  5,  6,  7,  3,  1,  4,  4,  0,  4,  7,
        3,  5,  6,  9, 10,  9, 10, 11,  9, 10, 11,  8])

In [11]:
d

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 6,
       6, 7, 7, 7, 8, 8, 8])

### Gibbs sampling inference
The model keeps track of the frequency that words in document d were assigned to topic t, "a[d,t]", and that vocabulary word w was assigned to topic t, "b[w,t]".  For convenience we also track "c[t]", the total number of words assigned to topic t.

We initialize the assignment vector "z[i]" to assign each word to a topic at random.

In [12]:
np.random.seed(0)

T = 2           # Only two topics ("HCI" and "Graph theory")
beta = .001     # LDA parameters: control sparsity of documents & topics
alpha = .001

a = np.zeros((D,T),dtype=int)   # total times a token in document d has been assigned to topic t
b = np.zeros((W,T),dtype=int)   # total times word w has been assigned to topic t
c = np.zeros((T,),dtype=int)    # total assignments to topic t

z = np.random.randint(T,size=N);  # assign every token to a topic at random
for n in range(N):
    a[d[n],z[n]] += 1             # and each document ID
    b[w[n],z[n]] += 1             # count up number for each word ID
    c[z[n]] += 1

Then, we can run "collapsed" Gibbs sampling for the LDA model, which samples a value for each z[i] in turn, holding the others fixed.  In practice, we update the sufficient statistics a,b,c to remove z[i]'s assignment, use them to compute p(z[i] | z[not i]), and then re-update a,b,c to reflect the new assignment z[i]:

In [13]:
def gibbs(maxIter):
    for it in range(maxIter):             # for each iteration 
        for i in range(N):                # run through all the words & sample z[i]:
            t = z[i];
            a[d[i],t] -= 1                # remove token i's assignment from our count vectors
            b[w[i],t] -= 1            
            c[t]      -= 1

            # Compute topic probability distribution given current counts
            probs = (beta + b[w[i],:])/(c[:] + beta*W) * (alpha + a[d[i],:])

            # Now, normalize and draw a sample from the distribution over topics:
            cumprobs = np.cumsum(probs); cumprobs /= cumprobs[-1] 
            t = np.where(cumprobs>np.random.rand())[0][0]

            z[i] = t;                      # now add this assignment back into our count vectors
            a[d[i],t] += 1
            b[w[i],t] += 1
            c[t]      += 1

In [14]:
# Helper function for visualizing the learning process
def print_topics():
    '''Print the top 8 words in each topic given the current assignments'''
    for t in range(T):
        isort = np.argsort(-b[:,t])  # find the most likely words for topic t
        xsort = b[isort,t]           # then print topic, % tokens explained, & top 8 words
        print('[{}] ({:.3f}) {}'.format(t, 1.*c[t]/N, list(words[isort[ww]] for ww in range(8))))

Now, let's just step through a few iterations of the Gibbs sampling inference process and see what the topics look like:

In [15]:
print_topics();       # before running any iterations: random assignment

[0] (0.414) ['human', 'system', 'interface', 'user', 'response', 'eps', 'survey', 'trees']
[1] (0.586) ['computer', 'user', 'system', 'time', 'trees', 'graph', 'interface', 'response']


Notice at this point the topics are "blended", with some words from the HCI papers and some from the graph papers.

In [16]:
gibbs(20)             # run a few iterations
print_topics();

[0] (0.483) ['system', 'human', 'interface', 'computer', 'user', 'eps', 'response', 'time']
[1] (0.517) ['trees', 'graph', 'response', 'time', 'survey', 'minors', 'user', 'human']


In [17]:
gibbs(180)            # run a bunch of iterations
print_topics();

[0] (0.690) ['system', 'user', 'human', 'interface', 'computer', 'response', 'time', 'eps']
[1] (0.310) ['trees', 'graph', 'minors', 'survey', 'human', 'interface', 'computer', 'user']


By now, we see that the HCI topic is more common (there are more documents that use it), and the words are now associated with each other in a more coherent way.

We can also visualize what's happening by looking at how many words are assigned to each topic in the documents:

In [18]:
for i in range(len(a)): print(str(a[i])+": "+" ".join(filter_text(docs[i])))

[3 0]: human machine interface abc computer applications
[6 0]: survey user opinion computer system response time
[4 0]: eps user interface management system
[4 0]: system human system engineering testing eps
[3 0]: relation user perceived response time error measurement
[0 1]: generation random binary ordered trees
[0 2]: intersection graph paths trees
[0 3]: graph minors widths trees well quasi ordering
[0 3]: graph minors survey


So, the first 5 documents are all assigned to topic 1; the last 4 all to topic 2.  We see a similar story for the vocabulary words:

In [19]:
for j in range(len(b)): print(str(b[j])+": "+words[j])

[2 0]: human
[2 0]: interface
[2 0]: computer
[3 0]: user
[4 0]: system
[2 0]: response
[2 0]: time
[2 0]: eps
[1 1]: survey
[0 3]: trees
[0 3]: graph
[0 2]: minors


Most vocabulary words are exclusively assigned to the topic most associated with their documents, although the word "survey" appears in both types of documents.

### Real data
Now we can look at how to do this on some real data:

In [20]:
# build vocab list (if necessary)
min_word_count = 5      # minimum length for an non-stopword
D,W,N = 0,0,0

# Data set is a few NYTimes articles from Jan 1 2000:
corpus = '../../../ml-data/topicmodel/example1/*.txt'

# run over documents, collecting words that occur:
vocab = {}
import glob
for filename in glob.glob(corpus):
    with open(filename,'rt') as fh:
        text = filter_text(fh.read())
        for word in text:
            if word in vocab: vocab[word] = vocab[word]+1
            else: vocab[word] = 1
            N += 1                 # count number of tokens (words in documents)
    D += 1                         # count # of documents

# now remove rare vocabulary:
remove = [w for w in vocab if vocab[w] <= min_word_count]
for w in remove: del vocab[w]

# now make vocab map to a unique index, rather than a count:
vocab = {w:i for i,w in enumerate(vocab)}   # map words to index
words = {vocab[w]:w for w in vocab}         # and an inverse map
W = len(vocab)   # count # of words

w,d = np.zeros((N,),dtype=int),np.zeros((N,),dtype=int)   # allocate storage for data
i = 0    
for j,filename in enumerate(glob.glob(corpus)):
    with open(filename,'rt') as fh:
        text = filter_text(fh.read())
        for word in text: 
            if word in vocab:
                d[i] = j
                w[i] = vocab[word]
                i += 1

N = i+1
w = w[:N]  # get rid of extra storage
d = d[:N]

In [21]:
np.random.seed(0)

T = 8           # Let's use 8 topics now
beta = .01      # LDA parameters: control sparsity of documents & topics
alpha = .1

a = np.zeros((D,T),dtype=int)   # total times a token in document d has been assigned to topic t
b = np.zeros((W,T),dtype=int)   # total times word w has been assigned to topic t
c = np.zeros((T,),dtype=int)    # total assignments to topic t

z = np.random.randint(T,size=N);  # assign every token to a topic at random
for n in range(N):
    a[d[n],z[n]] += 1             # and each document ID
    b[w[n],z[n]] += 1             # count up number for each word ID
    c[z[n]] += 1

In [22]:
print_topics();       # before running any iterations: random assignment

[0] (0.124) ['his', 'said', 'but', 'new', 'they', 'year', 'who', 'had']
[1] (0.124) ['said', 'but', 'his', 'they', 'new', 'year', 'who', 'not']
[2] (0.125) ['said', 'his', 'new', 'but', 'they', 'who', 'not', 'year']
[3] (0.126) ['his', 'said', 'but', 'new', 'they', 'who', 'not', 'had']
[4] (0.127) ['said', 'his', 'new', 'but', 'not', 'who', 'year', 'they']
[5] (0.125) ['said', 'his', 'new', 'but', 'they', 'not', 'had', 'year']
[6] (0.125) ['said', 'his', 'new', 'who', 'but', 'they', 'year', 'not']
[7] (0.124) ['his', 'said', 'new', 'but', 'year', 'who', 'had', 'not']


We can see that initially, the topics' word distributions are all about the same, with the most common words being most probable.

In [23]:
gibbs(20)             # after a few iterations:
print_topics();       

[0] (0.106) ['about', 'their', 'century', 'how', '000', 'who', 'war', 'work']
[1] (0.116) ['who', 'but', 'many', 'people', 'there', 'they', 'into', 'percent']
[2] (0.131) ['more', 'but', 'not', 'one', 'their', 'will', 'then', 'they']
[3] (0.110) ['said', 'y2k', 'new', '2000', 'problems', 'system', 'other', 'states']
[4] (0.125) ['new', 'times', 'year', 'millennium', 'york', 'nyt', 'news', 'putin']
[5] (0.128) ['said', 'will', 'not', 'year', 'its', 'been', 'there', 'had']
[6] (0.124) ['his', 'when', 'who', 'home', 'him', 'coach', 'been', 'sports']
[7] (0.160) ['said', 'they', 'had', 'who', 'one', 'team', 'out', 'would']


In [24]:
gibbs(180)            # after a bunch of iterations
print_topics();

[0] (0.112) ['her', 'american', 'world', 'people', 'she', 'century', '000', 'life']
[1] (0.102) ['they', 'many', 'more', 'their', 'who', 'there', 'people', 'them']
[2] (0.190) ['his', 'will', 'one', 'new', 'who', 'out', 'like', 'more']
[3] (0.087) ['y2k', 'said', 'new', 'year', 'problems', 'computer', 'not', 'system']
[4] (0.118) ['new', 'times', 'year', 'millennium', 'nyt', '2000', 'york', 'news']
[5] (0.173) ['said', 'but', 'had', 'they', 'all', 'not', 'been', 'when']
[6] (0.084) ['his', 'when', 'him', 'been', 'about', 'sports', 'who', 'bowl']
[7] (0.133) ['they', 'game', 'team', 'season', 'but', 'year', 'players', 'said']


Now, the topics have resolved into something more meaningful, approximately grouped by subject area.

### Uses of LDA